In [2]:
from transformers import pipeline

### Task 1, 2 - Download model

In [3]:
roberta = pipeline(task="fill-mask", model="FacebookAI/xlm-roberta-base")

Some weights of the model checkpoint at FacebookAI/xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [51]:
distilbert = pipeline(task="fill-mask", model="distilbert/distilbert-base-multilingual-cased")

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [76]:
longformer = pipeline(task="fill-mask", model="sdadas/polish-longformer-large-4096")

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

KeyboardInterrupt: 

### Task 3 - See if model understands Polish cases

In [62]:
cases = [
    "mianownik",
    "dopełniacz",
    "celownik",
    "biernik",
    "narzędnik",
    "miejscownik",
    "wołacz"
]

queries_roberta = [
    "<mask> jest największym eksporterem wina.",
    "Bez <mask> nie damy rady dokończyć projektu",
    "Dałem <mask> dziś wolne. Bardzo się ucieszył.",
    "Zastąpiłem dziś <mask> na nocnej zmianie.",
    "Poszliśmy dziś z <mask> na plażę.",
    "Polska to najciekawszy kraj w <mask>",
    "Szanowny <mask>, chciałbym prosić Pana o przysługę."
]

roberta

In [64]:
for query, case in zip(queries_roberta,cases):
    print(f"{case} : {roberta(query)[0]['token_str']}")

mianownik : Polska
dopełniacz : niego
celownik : mu
biernik : pracę
narzędnik : dziećmi
miejscownik : Europie
wołacz : Pan


The model fails at predicting the last case - wołacz - correctly

In [ ]:
for query, case in zip(queries_roberta,cases):
    print(f"{case} : {longformer(query)[0]['token_str']}")

TTODO

In [65]:
queries_distilbert = [q.replace("<mask>","[MASK]") for q in queries_roberta]

for query, case in zip(queries_distilbert,cases):
    print(f"{case} : {distilbert(query)[0]['token_str']}")

mianownik : Miasto
dopełniacz : tego
celownik : było
biernik : pracował
narzędnik : преко
miejscownik : Europie
wołacz : ##m


The model makes noticable errors and even fails to detect polish language

### Task 4 - Long-range relationships

In [66]:
query = "Zaprosiła mnie do kina. Oboje jesteśmy introwertykami ale myślałem, że za jakiś czas, w końcu się przemogę i to ja zrobię pierwszy ruch. Nie spodziewałem się tego z <mask> strony."

distilbert(query)[0]['token_str']

'jej'

Model is able to recognize subject gender even in long range.

In [ ]:
longformer(query)[0]['token_str']

TODO

In [67]:
distilbert(query.replace("<mask>","[MASK]"))[0]['token_str']

'jednej'

The model uses completely incorrect word that only fits into a very local context



---

### Task 5 - real world knowledge

In [68]:
query = "Pierwszy miesiąc w roku nazywa się <mask>."

roberta(query)[0]['token_str']

'poniedziałek'

In [ ]:
longformer(query)[0]['token_str']

TODO

In [69]:
distilbert(query.replace("<mask>","[MASK]"))[0]['token_str']

'rok'

Models displays poor world knowledge

### Task 6 - zero-shot learning

In [70]:
queries_roberta = ["Analizując emocje w tekście 'Ten film był niesamowity. Naprawdę świetnie się bawiłem' możemy stwierdzić, że wypowiedź ta ma <mask> charakter",
                   "Analizując emocje w tekście 'Ten film był okropny. Myślałem, że wyjdę w połowie' możemy stwierdzić, że wypowiedź ta ma <mask> charakter",
                   "Analizując emocje w tekście 'Ten film był niezły. Myślę, że godny polecenia' możemy stwierdzić, że wypowiedź ta ma <mask> charakter",
                   "Analizując emocje w tekście 'Ten film był straszny. Nigdy więcej go nie obejrzę' możemy stwierdzić, że wypowiedź ta ma <mask> charakter",
                   "Analizując emocje w tekście 'Ten film zupełnie mi się nie podobał. Był tragiczny.' możemy stwierdzić, że wypowiedź ta ma <mask> charakter",]

In [73]:
for query, case in zip(queries_roberta,cases):
    print(roberta(query)[0]['token_str'])

pozytywny
pozytywny
pozytywny
pozytywny
pozytywny


Zero-shot learning returns many false positives

In [ ]:
for query, case in zip(queries_roberta,cases):
    print(roberta(query)[0]['token_str'])

TODO

In [74]:
queries_distilbert = [q.replace("<mask>","[MASK]") for q in queries_roberta]

for query, case in zip(queries_distilbert,cases):
    print(distilbert(query)[0]['token_str'])

돔
돔
돔
돔
돔


What is this even...